In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    FFF_Train_Decile8 = pd.read_pickle("FFF_Train_Decile8.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile8.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile8, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 14s 5ms/step - loss: 0.3007 - acc: 0.8949 - val_loss: 0.2633 - val_acc: 0.8966
Epoch 46/75
3055/3055 [==============================] - 14s 4ms/step - loss: 0.3006 - acc: 0.8923 - val_loss: 0.2628 - val_acc: 0.8979
Epoch 47/75
3055/3055 [==============================] - 14s 4ms/step - loss: 0.3036 - acc: 0.8930 - val_loss: 0.2630 - val_acc: 0.9005
Epoch 48/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2963 - acc: 0.8946 - val_loss: 0.2603 - val_acc: 0.8979
Epoch 49/75
3055/3055 [==============================] - 14s 4ms/step - loss: 0.2888 - acc: 0.8995 - val_loss: 0.2594 - val_acc: 0.8979
Epoch 50/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.3043 - acc: 0.8920 - val_loss: 0.2582 - val_acc: 0.8979
Epoch 51/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2932 - acc: 0.8959 - val_loss: 0.2594 - val_acc: 0.9005
Epoch 52/75
3055/3055 [==============================] - 14s


Epoch 00037: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 38/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2517 - acc: 0.9038 - val_loss: 0.2172 - val_acc: 0.9071
Epoch 39/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2480 - acc: 0.9008 - val_loss: 0.2173 - val_acc: 0.9071
Epoch 40/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2570 - acc: 0.8959 - val_loss: 0.2171 - val_acc: 0.9071
Epoch 41/75
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2551 - acc: 0.8972 - val_loss: 0.2170 - val_acc: 0.9071
Epoch 42/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2473 - acc: 0.9018 - val_loss: 0.2169 - val_acc: 0.9071
Epoch 43/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2518 - acc: 0.8989 - val_loss: 0.2170 - val_acc: 0.9071
Epoch 44/75
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2505 - acc: 0.9028 - v

Epoch 31/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2689 - acc: 0.8995 - val_loss: 0.2308 - val_acc: 0.8979
Epoch 32/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2711 - acc: 0.8966 - val_loss: 0.2313 - val_acc: 0.9005
Epoch 33/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2653 - acc: 0.8953 - val_loss: 0.2314 - val_acc: 0.8992
Epoch 34/100
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2625 - acc: 0.8982 - val_loss: 0.2307 - val_acc: 0.9005
Epoch 35/100
3055/3055 [==============================] - 15s 5ms/step - loss: 0.2729 - acc: 0.8946 - val_loss: 0.2312 - val_acc: 0.9031
Epoch 36/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2673 - acc: 0.9018 - val_loss: 0.2320 - val_acc: 0.9005

Epoch 00036: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 37/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2587 - acc: 0.9011 - val_loss: 0

3055/3055 [==============================] - 13s 4ms/step - loss: 0.2636 - acc: 0.8972 - val_loss: 0.2298 - val_acc: 0.9018
Epoch 91/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2683 - acc: 0.9002 - val_loss: 0.2296 - val_acc: 0.9018
Epoch 92/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2728 - acc: 0.8936 - val_loss: 0.2294 - val_acc: 0.9018
Epoch 93/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2597 - acc: 0.8966 - val_loss: 0.2296 - val_acc: 0.9018
Epoch 94/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2653 - acc: 0.9002 - val_loss: 0.2291 - val_acc: 0.9018
Epoch 95/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2742 - acc: 0.8913 - val_loss: 0.2299 - val_acc: 0.9018
Epoch 96/100
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2600 - acc: 0.8985 - val_loss: 0.2300 - val_acc: 0.9018
Epoch 97/100
3055/3055 [==============================

...

3055/3055 [==============================] - 14s 5ms/step - loss: 0.2644 - acc: 0.8972 - val_loss: 0.2249 - val_acc: 0.9071
Epoch 21/50
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2558 - acc: 0.9005 - val_loss: 0.2232 - val_acc: 0.9084
Epoch 22/50
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2631 - acc: 0.8949 - val_loss: 0.2193 - val_acc: 0.9084
Epoch 23/50
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2612 - acc: 0.8995 - val_loss: 0.2281 - val_acc: 0.9031
Epoch 24/50
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2597 - acc: 0.8982 - val_loss: 0.2242 - val_acc: 0.9097

Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 25/50
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2544 - acc: 0.8946 - val_loss: 0.2201 - val_acc: 0.9071
Epoch 26/50
3055/3055 [==============================] - 14s 4ms/step - loss: 0.2536 - acc: 0.8998 - val_loss: 0.2

Epoch 28/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3185 - acc: 0.8969 - val_loss: 0.2863 - val_acc: 0.8914
Epoch 29/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3202 - acc: 0.8939 - val_loss: 0.2834 - val_acc: 0.8940
Epoch 30/125
3055/3055 [==============================] - 15s 5ms/step - loss: 0.3143 - acc: 0.8972 - val_loss: 0.2812 - val_acc: 0.8927
Epoch 31/125
3055/3055 [==============================] - 14s 4ms/step - loss: 0.3146 - acc: 0.8946 - val_loss: 0.2800 - val_acc: 0.8914
Epoch 32/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.3108 - acc: 0.8946 - val_loss: 0.2805 - val_acc: 0.8940
Epoch 33/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3109 - acc: 0.8956 - val_loss: 0.2792 - val_acc: 0.8927
Epoch 34/125
3055/3055 [==============================] - 12s 4ms/step - loss: 0.3112 - acc: 0.8998 - val_loss: 0.2777 - val_acc: 0.8927
Epoch 35/125
3055/3055 [=================

Epoch 88/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2688 - acc: 0.8962 - val_loss: 0.2296 - val_acc: 0.9005
Epoch 89/125
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2684 - acc: 0.8982 - val_loss: 0.2303 - val_acc: 0.8992
Epoch 90/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2663 - acc: 0.8985 - val_loss: 0.2296 - val_acc: 0.9005
Epoch 91/125
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2658 - acc: 0.8962 - val_loss: 0.2321 - val_acc: 0.8992
Epoch 92/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2741 - acc: 0.8969 - val_loss: 0.2300 - val_acc: 0.9005
Epoch 93/125
3055/3055 [==============================] - 13s 4ms/step - loss: 0.2706 - acc: 0.8989 - val_loss: 0.2298 - val_acc: 0.9005

Epoch 00093: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 94/125
3055/3055 [==============================] - 14s 5ms/step - loss: 0.2652 - acc: 0.8943 - val_loss: 0

3055/3055 [==============================] - 9s 3ms/step - loss: 0.2401 - acc: 0.8969 - val_loss: 0.2189 - val_acc: 0.9084
Epoch 56/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2403 - acc: 0.9008 - val_loss: 0.2189 - val_acc: 0.9084
Epoch 57/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2492 - acc: 0.8969 - val_loss: 0.2188 - val_acc: 0.9084
Epoch 58/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2394 - acc: 0.9041 - val_loss: 0.2188 - val_acc: 0.9084
Epoch 59/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2437 - acc: 0.9041 - val_loss: 0.2190 - val_acc: 0.9084
Epoch 60/100
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2433 - acc: 0.9038 - val_loss: 0.2188 - val_acc: 0.9084
Epoch 61/100
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2401 - acc: 0.9015 - val_loss: 0.2187 - val_acc: 0.9084
Epoch 62/100
3055/3055 [==============================] 

3055/3055 [==============================] - 9s 3ms/step - loss: 0.2603 - acc: 0.8956 - val_loss: 0.2247 - val_acc: 0.8992
Epoch 45/50
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2643 - acc: 0.8962 - val_loss: 0.2248 - val_acc: 0.9018
Epoch 46/50
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2648 - acc: 0.9002 - val_loss: 0.2227 - val_acc: 0.9045
Epoch 47/50
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2604 - acc: 0.8972 - val_loss: 0.2294 - val_acc: 0.9045

Epoch 00047: ReduceLROnPlateau reducing learning rate to 5.0000002374872565e-05.
Epoch 48/50
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2624 - acc: 0.8969 - val_loss: 0.2251 - val_acc: 0.9045
Epoch 49/50
3055/3055 [==============================] - 9s 3ms/step - loss: 0.2614 - acc: 0.8953 - val_loss: 0.2233 - val_acc: 0.9045
Epoch 50/50
3055/3055 [==============================] - 10s 3ms/step - loss: 0.2585 - acc: 0.8956 - val_loss: 0.2225 - 

In [ ]:
0.93484